# Extract data using an API
Use a public application program interface (API) to download some data.  Many web sites have data available through APIs, though to prevent abuse, you many need to register or pay a fee.  The US government is far less restrictive and has many sets of data freely available through https://data.gov 

In this notebook, lets download some data from the [EPA's Toxics Release Inventory (TRI)](https://www.epa.gov/enviro/tri-overview), a useful source of data on reporting how toxic chemicals are managed.  Like a lot of things, there are many resources available by searching the internet.  I'm drawing on inspiration (and code) from [this project](https://github.com/kperry2215/pull_data_from_EPA_Envirofacts_API)

As with any exploratory data analysis, you should get familiar with the data.  [This link is a good resource](https://www.epa.gov/toxics-release-inventory-tri-program/factors-consider-when-using-toxics-release-inventory-data)

It helps to start with some questions that come to mind.  Here, let's ask the data:  

 - What are the major ways chemicals are disposed?  
 - What are the top chemicals burned for energy recovery? Are there trends over time?

In [ ]:
import holoviews as hv
import hvplot.pandas
import pandas as pd
import io
import requests
from bs4 import BeautifulSoup # parse some xml returned 

Define a function to take in the names of tables and other filters, and return a pandas dataframe.  The call to the API is constructed according to the rules described here: https://www.epa.gov/enviro/envirofacts-data-service-api


In [11]:
def TRI_Query(table_name1,table_name2,table_name3,state=None, county=None,
                        area_code=None, year=None,chunk_size=100000):
    base_url='https://data.epa.gov/efservice/'
    output_format='CSV'
    query = base_url
    query+=table_name1+'/'
    #Add in the state qualifier, if the desired_state variable is named
    if state:
        query+='state_abbr/=/'+state+'/'
    #Add in the county qualifier, if the desired_county variable is named
    if county:
        query+='county_name/'+county+'/'
    #Add in the area code qualifier, if the desired_area_code variable is named
    if area_code:
        query+='zip_code/'+str(area_code)+'/'
    #Add in the next table name and year qualifier, if the desired_year variable is named
    query += table_name2+'/'
    if year:
        if type(year) is list:
            query+='reporting_year/'+str(year[0])+'/'+str(year[1])+'/'
        else:        
            query+='reporting_year/'+str(year)+'/'
    #add the third table
    query += table_name3+'/'
    count_query = query+'count/'
    
    count_xml = requests.get(count_query).content
    
    nrows= int(BeautifulSoup(count_xml).find('requestrecordcount').contents[0])
    
    #Add in the desired output format to the query
    csv_query = query+ output_format

    
    #Return the completed query
    print('Start download of {} rows'.format(nrows))
    s=requests.get(csv_query).content
    dataframe=pd.read_csv(io.StringIO(s.decode('utf-8')), engine='python',
                      encoding='utf-8', error_bad_lines=False)
    print('Downloaded data {} of {} rows'.format(dataframe.shape[0],nrows))
    while dataframe.shape[0] < nrows:
        new_query = query + 'rows/'+str(dataframe.shape[0])+':'+str(dataframe.shape[0]+chunk_size)+'/'
        csv_query = new_query+ output_format
        s=requests.get(csv_query).content
        dataframe = dataframe.append(pd.read_csv(io.StringIO(s.decode('utf-8')), engine='python',
                      encoding='utf-8', error_bad_lines=False))
        print('Downloaded data {} of {} rows'.format(dataframe.shape[0],nrows))
        
    return dataframe




We will join three tables from Envirofacts: *TRI_REPORTING_FORM* contains the data from the Toxic Chemical Release Inventory Reporting Form submitted each year, *TRI_FACILITY* contains the name and location of the facility producing the waste, *TRI_TRANSFER_QTY* contains the data on the quantities of the chemical (in pounds) moved from the reporting facility.  Some more information on the TRI form codes can be found here: https://www.epa.gov/sites/production/files/documents/ry2012rfi.pdf


Lets read in the data using the function defined above, and, due to the sheer size, limit the data to the past 10 years:

In [42]:
#Declare the names of the tables that we want to pull
table_name2='TRI_REPORTING_FORM'
table_name1='TRI_FACILITY'
table_name3 = 'TRI_TRANSFER_QTY'

# call the function with the table names and filter criteria
tri_df =TRI_Query(table_name1,table_name2,table_name3,year=['>',2008])


Start download of 1834391 rows
Downloaded data 100001 of 1834391 rows
Downloaded data 200002 of 1834391 rows
Downloaded data 300003 of 1834391 rows
Downloaded data 400004 of 1834391 rows
Downloaded data 500005 of 1834391 rows
Downloaded data 600006 of 1834391 rows
Downloaded data 700007 of 1834391 rows
Downloaded data 800008 of 1834391 rows
Downloaded data 900009 of 1834391 rows
Downloaded data 1000010 of 1834391 rows
Downloaded data 1100011 of 1834391 rows
Downloaded data 1200012 of 1834391 rows
Downloaded data 1300013 of 1834391 rows
Downloaded data 1400014 of 1834391 rows
Downloaded data 1500015 of 1834391 rows
Downloaded data 1600016 of 1834391 rows
Downloaded data 1700017 of 1834391 rows
Downloaded data 1800018 of 1834391 rows
Downloaded data 1834391 of 1834391 rows


In [43]:
tri_df.shape

(1834391, 114)

Inspect a random sample of the data to make sure it is what we expect

In [44]:
tri_df.groupby(['TRI_REPORTING_FORM.CAS_CHEM_NAME',
                 'TRI_FACILITY.STANDARDIZED_PARENT_COMPANY',
                 'TRI_TRANSFER_QTY.TYPE_OF_WASTE_MANAGEMENT'])['TRI_TRANSFER_QTY.TOTAL_TRANSFER'].sum().sample(5)

TRI_REPORTING_FORM.CAS_CHEM_NAME  TRI_FACILITY.STANDARDIZED_PARENT_COMPANY  TRI_TRANSFER_QTY.TYPE_OF_WASTE_MANAGEMENT
ZINC COMPOUNDS                    HUBBELL HOLDING CORP                      M41                                             433.00
BENZENE                           HOLCIM (US) INC                           M92                                            1022.70
NICKEL                            ARCONIC INC                               M94                                          100526.28
DIISOCYANATES                     THE SHERWIN-WILLIAMS CO                   M69                                          181066.00
POLYCYCLIC AROMATIC COMPOUNDS     EXPERA SPECIALTY SOLUTIONS LLC            M64                                               8.70
Name: TRI_TRANSFER_QTY.TOTAL_TRANSFER, dtype: float64

Save the downloaded data to explore offline as a plain text file

In [45]:
tri_df.to_csv('TRI_DATA.tsv',sep='\t') #there are commas in the text so use tab delimiters

Create a lookup dictionary to replace values for the codes

In [46]:
wm_dict = {'P91':'Waste water treatment',
           'M56':'Energy Recovery',
           'M50':'Incineration/Thermal Treatment',
           'M64':'Other Landfills',
           'M24':'Metals Recovery',
           'M26':'Other Reuse or Recovery',
           'M41':'Solidification/Stabilization - Metals and Metal Category Compounds only',
           'M90':'Other Off-Site Management',
           'M61':'Wastewater Treatment (Excluding POTW)',
           'M93':'Transfer to Waste Broker - Recycling ',
           'M92':'Transfer to Waste Broker - Energy Recovery',
           'M94':'Transfer to Waste Broker - Disposal',
           'M20':'Solvents/Organics Recovery',
           'M99':'Unknown',
           'M54':'Incineration/Insignificant Fuel Value ',
           'M95':'Transfer to Waste Broker - Waste Treatment',
           'M62':'Wastewater Treatment (Excluding POTW) - Metals and Metal Category Compounds only',
           'M79':'Other Land Disposal',
           'M65':'RCRA Subtitle C Landfills',
           'M69':'Other Waste Treatment',
           'M40':'Solidification/Stabilization',
           'M73':'Land Treatment',
           'M10':'Storage Only',
           'M81':'Underground Injection to Class I Wells',
           'M66':'Subtitle C Surface Impoundment',
           'M67':'Other Surface Impoundments',
           'M28':'Acid Regeneration',
           'M82':'Underground Injection to Class II- V Wells',
           'M72':'Landfill/Disposal Surface Impoundment'
          }

In [47]:
# do the replacement:
tri_df.replace({'TRI_TRANSFER_QTY.TYPE_OF_WASTE_MANAGEMENT':wm_dict},inplace=True)

In [69]:
type_df = tri_df[['TRI_TRANSFER_QTY.TOTAL_TRANSFER',
                  'TRI_TRANSFER_QTY.TYPE_OF_WASTE_MANAGEMENT']].groupby('TRI_TRANSFER_QTY.TYPE_OF_WASTE_MANAGEMENT').sum()
type_df['% of total'] = type_df['TRI_TRANSFER_QTY.TOTAL_TRANSFER']/type_df['TRI_TRANSFER_QTY.TOTAL_TRANSFER'].sum()*100
type_df.sort_values('% of total',ascending=False).head(5)

,TRI_TRANSFER_QTY.TOTAL_TRANSFER,% of total
TRI_TRANSFER_QTY.TYPE_OF_WASTE_MANAGEMENT,,
Metals Recovery,1.455281e+10,39.551655
Transfer to Waste Broker - Recycling,4.026914e+09,10.944356
Energy Recovery,3.516161e+09,9.556230
Other Reuse or Recovery,2.526311e+09,6.866014
Waste water treatment,2.439617e+09,6.630397


In [49]:
subset = tri_df.loc[tri_df['TRI_TRANSFER_QTY.TYPE_OF_WASTE_MANAGEMENT'].isin(['Energy Recovery',
                                                                                'Transfer to Waste Broker - Energy Recovery'])]
df = pd.pivot_table(subset,values='TRI_TRANSFER_QTY.TOTAL_TRANSFER',
                    index=['TRI_REPORTING_FORM.REPORTING_YEAR'],
                    columns=['TRI_REPORTING_FORM.CAS_CHEM_NAME'], aggfunc=sum, fill_value=0)

In [51]:
top_chems = df.sum(axis='rows').sort_values(ascending=False).head(10)
top_chems

TRI_REPORTING_FORM.CAS_CHEM_NAME
METHANOL                  9.244396e+08
TOLUENE                   7.382267e+08
XYLENE (MIXED ISOMERS)    5.263581e+08
CERTAIN GLYCOL ETHERS     1.224165e+08
STYRENE                   1.214858e+08
ETHYLENE                  1.124619e+08
ETHYLBENZENE              1.058655e+08
N-BUTYL ALCOHOL           9.846949e+07
METHYL ISOBUTYL KETONE    8.626281e+07
ACETONITRILE              7.949454e+07
dtype: float64

In [52]:
df.loc[:,top_chems.index].hvplot(width=800,height=400,logy=True)

:NdOverlay   [Variable]
   :Curve   [TRI_REPORTING_FORM.REPORTING_YEAR]   (value)